In [11]:
import nltk
import numpy as np
import matplotlib.pyplot as plt

import gensim.downloader as api
from gensim.models import word2vec
from sklearn.manifold import TSNE


In [12]:
# notebook hparams
corpus_path = r"..\barrons_333_corpus.txt"
W2V_SIZE = 100    # Word vector dimensionality
W2V_WINDOW = 30   # Context window size
W2V_MIN_COUNT = 1    # Minimum word count
W2V_EPOCHS = 50    # w2v model training iters

np.set_printoptions(suppress=True)

In [3]:
# extracting gensim pretrained model & corpora info
def get_gensim_pretrained_info(entity, desc_len=None):
    """
    :param entity: either 'corpora' or 'models'
    :param desc_len: description length of each entity, entire description is printed if this is None
    :return: None
    """
    info = api.info()
    for entity_name, entity_data in sorted(info[entity].items()):
        print(f"{entity_name:<40} {entity_data.get('num_records', -1)} records: "
              f"{entity_data['description'][:desc_len] + '...'}")

In [4]:
# tokenize a corpus using nltk
def nltk_corpus_tokenizer(corpus):
    # tokenize sentences in corpus
    wpt = nltk.WordPunctTokenizer()
    tokenized_corpus = wpt.tokenize(corpus)
    return tokenized_corpus

In [5]:
# train a w2v model on a given corpus
def train_w2v_model(corpus, size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, iter=W2V_EPOCHS, workers=4):
    logging.info(f'word2vec model training started with params {size, window, min_count, iter, workers}')
    w2v_model = word2vec.Word2Vec(corpus,
                                  size=size,
                                  window=window,
                                  min_count=min_count,
                                  iter=iter,
                                  workers=workers)
    logging.info(f'word2vec model training completed..')
    return w2v_model

## getting w2v vectors

In [9]:
get_gensim_pretrained_info('models', desc_len=80)
# w2v_model = train_w2v_model(tokenized_corpus)

__testing_word2vec-matrix-synopsis       -1 records: [THIS IS ONLY FOR TESTING] Word vecrors of the movie matrix....
conceptnet-numberbatch-17-06-300         1917247 records: ConceptNet Numberbatch consists of state-of-the-art semantic vectors (also known...
fasttext-wiki-news-subwords-300          999999 records: 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt...
glove-twitter-100                        1193514 records: Pre-trained vectors based on  2B tweets, 27B tokens, 1.2M vocab, uncased (https:...
glove-twitter-200                        1193514 records: Pre-trained vectors based on 2B tweets, 27B tokens, 1.2M vocab, uncased (https:/...
glove-twitter-25                         1193514 records: Pre-trained vectors based on 2B tweets, 27B tokens, 1.2M vocab, uncased (https:/...
glove-twitter-50                         1193514 records: Pre-trained vectors based on 2B tweets, 27B tokens, 1.2M vocab, uncased (https:/...
glove-wiki-gigaword-100         

In [6]:
with open(corpus_path, 'r') as f:
    barrons_corpus = f.read()
    
tokenized_corpus = nltk_corpus_tokenizer(barrons_corpus)
print(f'length of raw corpus {len(tokenized_corpus)}')

334


In [21]:
def filter_corpus_vocab(w2v_model, tokenized_corpus):
    trained_words = list()
    untrained_words = list()
    model_vocab = w2v_model.vocab
    for _word in tokenized_corpus:
        if _word in model_vocab:
            trained_words.append(_word)
        else:
            untrained_words.append(_word)
    
    print(f"w2v model doesn't have {len(untrained_words)} words: {untrained_words}")
    return trained_words

In [ ]:
# load pre trained model
w2v_model = api.load("glove-wiki-gigaword-50")
tokenized_corpus = _filter_corpus_vocab(w2v_model, tokenized_corpus)

In [13]:
# sample runs
for _word in ["stop", "woman", "man", "abate", "india"]:
    neighbors = w2v_model.most_similar(_word, topn=5)
    print(neighbors)

[('stopping', 0.8896316885948181), ('stopped', 0.8306300640106201), ('trying', 0.8275439739227295), ('tried', 0.8172997832298279), ('stops', 0.8129743933677673)]
[('girl', 0.9065280556678772), ('man', 0.8860336542129517), ('mother', 0.8763703107833862), ('her', 0.86131352186203), ('boy', 0.8596119284629822)]
[('woman', 0.8860337734222412), ('boy', 0.8564431071281433), ('another', 0.8452839851379395), ('old', 0.8372182846069336), ('one', 0.8276063203811646)]
[('caputo', 0.6675522923469543), ('washpost.com', 0.6115913391113281), ('levesque', 0.6013662815093994), ('subside', 0.5964425206184387), ('greef', 0.5947305560112)]
[('indian', 0.8648794889450073), ('pakistan', 0.8529723286628723), ('malaysia', 0.816650927066803), ('bangladesh', 0.8154239058494568), ('delhi', 0.8142766952514648)]


In [ ]:
wvs = w2v_model.wv[tokenized_corpus]
print(len(wvs), wvs.shape)

In [19]:
q = w2v_model.vocab.keys()


400000 True False


In [ ]:
tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=2)
T = tsne.fit_transform(wvs)

labels = tokenized_corpus
plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')